<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/baseline/notebooks/data_preprocessing_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.3 MB/s 
     |████████████████████████████████| 956 kB 25.4 MB/s 
     |████████████████████████████████| 956 kB 62.4 MB/s 
     |████████████████████████████████| 761 kB 59.4 MB/s 


In [2]:
# use gcs fuse to access google cloud storage
# https://stackoverflow.com/a/60450255/5755604
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# mount google cloud stoarge as drive
!mkdir gcs
!gcsfuse thesis-bucket-option-trade-classification gcs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  81838      0 --:--:-- --:--:-- --:--:-- 81838
OK
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.8_amd64.deb ...
Unpacking gcsfuse (0.41.8) ...
Setting up gcsfuse (0.41.8) ...
2022/10/28 14:49:31.040851 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount po

In [8]:
import numpy as np
import modin.pandas as pd
import modin.config as cfg
from modin.config import ProgressBar
from distributed import Client
cfg.Engine.put("dask")
ProgressBar.enable()

from tqdm.notebook import tqdm

In [4]:
from google.colab import auth
auth.authenticate_user()


In [6]:
client = Client()

file_path = "./gcs/data/preprocessed/matched_ise_quotes_2017.parquet"

df = pd.read_parquet(file_path)

In [7]:
df.head().T

,46508339,46508340,46508341,46508342,46508343
UNDERLYING_SYMBOL,UNG,UNG,UNG,UNG,UNG
QUOTE_DATETIME,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00
SEQUENCE_NUMBER,173631,67282495,134391223,1409459756,1476568620
ROOT,UNG,UNG,UNG,UNG,UNG
EXPIRATION,2017-01-20 00:00:00,2017-02-17 00:00:00,2017-02-17 00:00:00,2017-07-21 00:00:00,2017-07-21 00:00:00
STRK_PRC,7.0,7.0,13.0,10.0,14.0
OPTION_TYPE,C,C,C,C,C
TRADE_SIZE,2,2,2,1,1
TRADE_PRICE,1.67,1.67,0.13,0.71,0.1
BEST_BID,1.49,1.44,0.0,0.4,0.0


In [32]:
# quote rule
mid = 0.5 * (df['BEST_ASK'] - df['BEST_BID'])
df['pred_quote_rule'] = np.where(df['TRADE_PRICE'] > mid, 1, np.where(df['TRADE_PRICE'] < mid, -1, np.nan))

In [ ]:
# tick rule
df['pred_tick_test'] = np.where(df['TRADE_PRICE'] < df['price_all_lead'], -1.0, 1.0)

In [34]:
df.head().T

,46508339,46508340,46508341,46508342,46508343
UNDERLYING_SYMBOL,UNG,UNG,UNG,UNG,UNG
QUOTE_DATETIME,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00,2017-01-03 09:30:00
SEQUENCE_NUMBER,173631,67282495,134391223,1409459756,1476568620
ROOT,UNG,UNG,UNG,UNG,UNG
EXPIRATION,2017-01-20 00:00:00,2017-02-17 00:00:00,2017-02-17 00:00:00,2017-07-21 00:00:00,2017-07-21 00:00:00
STRK_PRC,7.0,7.0,13.0,10.0,14.0
OPTION_TYPE,C,C,C,C,C
TRADE_SIZE,2,2,2,1,1
TRADE_PRICE,1.67,1.67,0.13,0.71,0.1
BEST_BID,1.49,1.44,0.0,0.4,0.0


In [31]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(df['buy_sell'], df['pred_tick_test'])

print(acc)

0.5492322498115313


In [ ]:
# check against some stats from sub panel A.1 in Grauer et al
# TODO: convert to proper tests

# trade size
stats_trade_size = df['TRADE_SIZE'].agg(['mean','median','std'])

# moneyness; price underlying / strike
# TODO: Request price for underlyings?

# time to maturity
stats_time_to_maturity = (df['EXPIRATION'] - df['QUOTE_DATETIME']).dt.days
stats_time_to_maturity = stats_time_to_maturity.agg(['mean','median','std'])

# no of observations
stats_n = len(df)

# trade_size = quote size; TRADE_SIZE
stats_trades_with_quote_size_bid = df['bid_size_ex'].eq(df['TRADE_SIZE'])
stats_trades_with_quote_size_ask = df['ask_size_ex'].eq(df['TRADE_SIZE'])
# either ask or bid must be equal, but not both (XOR)
stats_trade_with_quote_size = (stats_trades_with_quote_size_bid ^ stats_trades_with_quote_size_ask).sum() / stats_n

# no of buys
stats_buy_trades = df['buy_sell'].ge(0).sum() / stats_n

# underlyings per day
# stats_underlyings_per_day = df.groupby(['UNDERLYING_SYMBOL','QUOTE_DATETIME']).count().agg(['mean','median','std'])